<a href="https://colab.research.google.com/github/AdnanMulla/ai-courses/blob/module-1/AI%20Agents%20and%20Agentic%20AI%20with%20Python%20%26%20Generative%20AI/Module%201/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'GEMINI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = api_key

In [2]:
from litellm import completion
from typing import List, Dict
import json

print("START")

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
      model="gemini/gemini-2.0-flash",
      messages=messages
    )
    return response.choices[0].message.content

code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"},
    # {"role": "user", "content": f"Can you please shorten the description?"}
]


response = generate_response(messages)
print(response)
print("END")

START
```python
def swap_keys_values(d):
    """Swaps the keys and values in a given dictionary.

    Args:
        d (dict): A dictionary with unique values.

    Returns:
        dict: A new dictionary with keys and values swapped.
             Returns an empty dictionary if the input dictionary is empty.

    Raises:
        TypeError: if the input is not a dictionary.
        ValueError: if the dictionary contains duplicate values.
    """

    if not isinstance(d, dict):
        raise TypeError("Input must be a dictionary.")

    if not d:
        return {}

    # Check for duplicate values
    values = list(d.values())
    if len(values) != len(set(values)):
        raise ValueError("Dictionary must have unique values.")

    return {value: key for key, value in d.items()}
```
END


In [ ]:
# Building a quasi-agent

# First Prompt

def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

function_description = ""
def get_user_input():
  print("What kind of function would you like to create?")
  print("Eg. Function that generate fibonacci sequence")
  print("Your description: ", end="")
  function_description = input().strip()

  print("Creating function based on following description: ")
  return function_description

function_description = get_user_input()

print("--")
print(function_description)
messages = [
    { "role": "system", "content": "You are a Python expert helping to develop a function."},
    {"role": "user", "content": f'Write a very small Python function that {function_description}. Output the function in a ```python code block```'}
]

initial_function = extract_code_block(generate_response(messages))
print("initial_function: ")
print(initial_function)

messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
messages.append({"role": "user", "content": "Add comprehensive documentation to this function, including description, parameters, return value, examples, and edge cases. Output the function in a ```python code block```."})

second_function = extract_code_block(generate_response(messages))

print("second_function: ")
print(second_function)

messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
messages.append({"role": "user", "content": "Add unittest test cases for this function, including tests for basic functionality, edge cases, error cases, and various input scenarios. Only generate the testcases. Output the code in a \`\`\`python code block\`\`\`."})

third_function = extract_code_block(generate_response(messages))

print("third_function")
print(third_function)

filename = function_description.lower()
filename = "".join(c for c in filename if c.isalnum() or c.isspace())
filename = filename.replace(" ", "_")[:30] + ".py"

with open(filename, "w") as f:
  f.write(second_function + "\n\n" + third_function)


What kind of function would you like to create?
Eg. Function that generate fibonacci sequence
Your description: check if number is prime
Creating function based on following description: 
--
check if number is prime
initial_function: 

def is_prime(n):
  """
  Checks if a number is prime.

  Args:
    n: The number to check.

  Returns:
    True if the number is prime, False otherwise.
  """
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True
second_function: 

def is_prime(n):
  """
  Checks if a number is prime.

  A prime number is a natural number greater than 1 that is not a product of two smaller natural numbers.
  In other words, a prime number has exactly two distinct positive divisors: 1 and itself.

  Args:
    n: The number to check for primality.  Must be an integer.

  Returns:
    True if the number is prime, False otherwise.

  Examples:
    is_prime(2) == True
    is_prime(10) == False
    is_prime(17) 